In [ ]:
import numpy as np
import pandas as pd 

import random
import sys
import matplotlib.pyplot as plt 
%matplotlib inline

sys.path.insert(1,'D:\\thesis-main\codes')
import time

from package.loadDb import *
from package.featureSelection_TENSORIZED import *
from package.featureSelection_TEMPORAL_PAGERANK import *
from package.featureSelection_SICE import *

from package.trmf import *
from package.trmf_regressor import *

from package.regressors import *

In [ ]:
dataset_name=
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target =load_energydata_complete_dataframe(address)
data_without_target = data_with_target.drop(['Appliances'],axis=1)
target=data_with_target.iloc[:,0]

In [ ]:
data_with_target

In [ ]:

data_with_target.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your data is already loaded in a DataFrame called data_with_target
# Example column lists
col1 = ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
        'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
        'RH_9']
col2 = ['T_out', 'RH_out', 'Windspeed', 'Visibility',
        'Tdewpoint', 'rv1', 'rv2']
col3 = ['Appliances', 'lights', 'Press_mm_hg']
col4=['Appliances','RH_5']

# Function to create scatter plots
def plot_columns(column_list, title):
    plt.figure(figsize=(14, 8))
    for column in column_list:
        plt.scatter(data_with_target.index, data_with_target[column], label=column)
    plt.xlabel('Date')
    plt.ylabel('Values')
    plt.title(f'Time Series Plot of {title}')
    plt.legend(loc='best', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.show()

# Plot for col1
plot_columns(col1, 'Temperature and Humidity')

# Plot for col2
plot_columns(col2, 'Environmental Factors')

# Plot for col3
plot_columns(col3, 'Appliances, Lights, and Pressure')
plot_columns(col4, 'Appliances, RH_5')



In [ ]:
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target =load_energydata_complete_dataframe(address)
data_without_target = data_with_target.drop(['Appliances'],axis=1)
target=data_with_target.iloc[:,0]

In [ ]:
#segment sizes
interval=600
testSize=int(.2*interval)
#energy of pca to compute number of components
tol=.89
testSize

In [ ]:
# trmf parameters..............................................
lags = [1,30]
# K = 4######### changes besed on pca energy
lambda_f = 1.
lambda_x = 1
lambda_w = 1.
alpha = 1000.
eta = 1.
max_iteration=10000
# num_comp=K

In [ ]:
num_comp_list=[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1]

In [ ]:
i

In [ ]:
from itertools import product

# Define parameter grids
lags_list = [[1], [1, 30]]
lambda_f_list = [0.1, 1., 10.]
lambda_x_list = [0.1, 1., 10.]
lambda_w_list = [0.1, 1., 10.]
alpha_list = [100, 1000, 10000]
eta_list = [0.1, 1., 10.]

# Store all RMSE results for each segment and parameter combination
results = []

counter = 0


# Iterate over all combinations of parameters except K
for lags, lambda_f, lambda_x, lambda_w, alpha, eta in product(lags_list, lambda_f_list, lambda_x_list, lambda_w_list, alpha_list, eta_list):
    i = 0
    for j in range(data_without_target.shape[0] // interval):
        start_index = j * interval
        end_index = start_index + interval

        K = num_comp_list[i]  # Each segment has different k
        data = data_without_target.to_numpy().T
        data = data[:, start_index:end_index]
        target_cut = target.iloc[start_index:end_index]

        T_train = data.shape[1]
        st = time.time()  # Start timer
        data_normalized = normalized_data(data, T_train, 0, normalize=True)

        model = trmf(lags, K, lambda_f, lambda_x, lambda_w, alpha, eta)
        model.fit(data_normalized, max_iter=max_iteration)
        et = time.time()  # End timer
        elapsed_time = et - st
        runtime.append(elapsed_time)

        # Get reduced data
        new_features = model.X.T
        rmse = xgboost_reg_error(new_features, target_cut, testSize)
        trmf_rmse_list.append(rmse)

        # Store the parameters and RMSE for this segment
        results.append({
            'segment': j,
            'K': K,
            'lags': lags,
            'lambda_f': lambda_f,
            'lambda_x': lambda_x,
            'lambda_w': lambda_w,
            'alpha': alpha,
            'eta': eta,
            'rmse': rmse,
            'runtime': elapsed_time
        })

        i += 1
        print(f"Iteration {i}, Segment {j}, K={K}, RMSE={rmse}, Time={elapsed_time}")
        print("----------------------------------------------------------------------------")

# Convert results to a DataFrame for easier analysis and visualization
import pandas as pd
results_df = pd.DataFrame(results)

# Example: Save results to a CSV file for later analysis
results_df.to_csv('trmf_tuning_results.csv', index=False)

# Now you can analyze or visualize the results


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure 'segment' is treated as a string or categorical variable
results_df['segment'] = results_df['segment'].astype(str)

# Convert list-type columns to tuples
results_df['lags'] = results_df['lags'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# Create a unique identifier for each parameter set
results_df['param_id'] = results_df.groupby(['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']).ngroup() + 1

# Get unique segments
unique_segments = results_df['segment'].unique()

# Plot size customization
plt.rcParams.update({'figure.figsize': (100, 30)})

# Iterate over each segment and create a separate plot
for segment in unique_segments:
    # Filter the DataFrame for the current segment
    segment_df = results_df[results_df['segment'] == segment]

    # Create a bar plot for the current segment
    plt.figure(figsize=(14, 10))
    sns.barplot(
        x='param_id', 
        y='rmse', 
        data=segment_df
    )

    # Set plot labels and title
    plt.xlabel('Parameter Set ID')
    plt.ylabel('RMSE')
    plt.title(f'RMSE for Segment {segment}')

    # Rotate x-axis labels if needed for better readability
    plt.xticks(rotation=0)

    # Show the plot
    plt.show()

# Create a separate legend figure to map numbers to parameter sets
unique_params = results_df.drop_duplicates('param_id').sort_values('param_id')
plt.figure(figsize=(12, len(unique_params) * 0.5))  # Adjust the figure size based on the number of sets

# Plot the mapping of parameter set ID to actual parameters
plt.table(cellText=unique_params[['param_id', 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']].values,
          colLabels=['ID', 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'],
          cellLoc = 'center', 
          loc='center')

plt.axis('off')  # Hide the axis
plt.title('Mapping of Parameter Set ID to Parameter Values')
plt.show()


In [ ]:
results_df['params'] = results_df[['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']].apply(lambda x: ', '.join(map(str, x)), axis=1)


In [ ]:

mean_rmse_df = results_df.groupby([ 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'])['rmse'].mean().reset_index()

mean_rmse_df.sort_values(by=['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'], ascending=[True, True, True, True, True, True]).to_excel("rmse_trmf_parameter_tuning.xlsx")

In [30]:
import pandas as pd
import numpy as np
from tsflex.features import FeatureCollection, FeatureDescriptor, FuncWrapper
from sklearn.feature_selection import SelectKBest, f_regression

# Load your data
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target = pd.read_csv(address)
data_with_target = data_with_target.reset_index()
data_with_target = data_with_target.rename(columns={"index": "id"})
data_with_target=data_with_target.iloc[:100]
# Separate features and target
data_without_target = data_with_target.drop(['Appliances'], axis=1)
target = data_with_target['Appliances']

data_without_target


,id,date,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0,2016-01-11 17:00:00,30,19.890000,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.530000,6.600000,733.500000,92.0,7.000000,63.000000,5.300000,13.275433,13.275433
1,1,2016-01-11 17:10:00,30,19.890000,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.560000,6.483333,733.600000,92.0,6.666667,59.166667,5.200000,18.606195,18.606195
2,2,2016-01-11 17:20:00,30,19.890000,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.500000,6.366667,733.700000,92.0,6.333333,55.333333,5.100000,28.642668,28.642668
3,3,2016-01-11 17:30:00,40,19.890000,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.400000,6.250000,733.800000,92.0,6.000000,51.500000,5.000000,45.410389,45.410389
4,4,2016-01-11 17:40:00,40,19.890000,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.400000,6.133333,733.900000,92.0,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,2016-01-12 08:50:00,0,19.890000,47.633333,19.0,45.090000,20.10,45.200000,19.100000,...,17.033333,47.266667,4.850000,742.216667,91.0,5.000000,40.000000,3.466667,39.865441,39.865441
96,96,2016-01-12 09:00:00,0,19.856667,47.063333,19.0,45.030000,20.10,45.200000,19.100000,...,17.000000,46.930000,4.800000,742.300000,91.0,5.000000,40.000000,3.400000,22.763723,22.763723
97,97,2016-01-12 09:10:00,0,19.790000,46.656667,19.0,44.966667,20.10,45.163333,19.100000,...,17.066667,46.790000,4.883333,742.383333,91.0,5.000000,38.166667,3.483333,20.504204,20.504204
98,98,2016-01-12 09:20:00,0,19.790000,46.333333,19.0,44.826667,20.10,45.090000,19.100000,...,17.033333,46.590000,4.966667,742.466667,91.0,5.000000,36.333333,3.566667,40.543512,40.543512


In [28]:
data_without_target.index

RangeIndex(start=0, stop=100, step=1, name='id')

In [31]:
from tsfresh import extract_features, select_features
X_extracted = extract_features(data_without_target, column_id='id', column_sort='date')
X_selected = select_features(X_extracted, target)

Feature Extraction:   0%|          | 0/10 [00:00<?, ?it/s]